In [1]:
sc #this is accessibe directly in the notebook

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
datasetPath="datasets/wine.data"

In [4]:
rawData = sc.textFile(datasetPath)

This is older mllib

In [5]:
rawData

datasets/wine.data MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [6]:
rawData.take(5) #equivalent to dataframe.head()

['1,14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065',
 '1,13.2,1.78,2.14,11.2,100,2.65,2.76,.26,1.28,4.38,1.05,3.4,1050',
 '1,13.16,2.36,2.67,18.6,101,2.8,3.24,.3,2.81,5.68,1.03,3.17,1185',
 '1,14.37,1.95,2.5,16.8,113,3.85,3.49,.24,2.18,7.8,.86,3.45,1480',
 '1,13.24,2.59,2.87,21,118,2.8,2.69,.39,1.82,4.32,1.04,2.93,735']

# columns
- 1st column in is the label , have values 1,2,3 and represents 3 cultivators of wine in italy
- Alcohol,malic acid,ash,magnesium,hue,color intensity,residual sugar etc

In [14]:
from pyspark.mllib.regression import LabeledPoint
#labelPoint expect label and features assocaited with it
#this return the dataset in the desiered format for ml model
def parsePoint(line):
    values = [float(x) for x in line.split(',')]
    #o is the lable and rest of it are the features
    return LabeledPoint(values[0],values[:])

In [15]:
#mapping the rawData to parsePoint
parsedData = rawData.map(parsePoint)

In [16]:
parsedData.take(5)

[LabeledPoint(1.0, [1.0,14.23,1.71,2.43,15.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0]),
 LabeledPoint(1.0, [1.0,13.2,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.4,1050.0]),
 LabeledPoint(1.0, [1.0,13.16,2.36,2.67,18.6,101.0,2.8,3.24,0.3,2.81,5.68,1.03,3.17,1185.0]),
 LabeledPoint(1.0, [1.0,14.37,1.95,2.5,16.8,113.0,3.85,3.49,0.24,2.18,7.8,0.86,3.45,1480.0]),
 LabeledPoint(1.0, [1.0,13.24,2.59,2.87,21.0,118.0,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735.0])]

- Split the data set into train and test

In [18]:
(train,test)=parsedData.randomSplit([0.7,0.3])

## Model Building

In [61]:
from pyspark.mllib.tree import DecisionTree

model = DecisionTree.trainClassifier(train,
                                     numClasses=4,
                                     categoricalFeaturesInfo={},
                                     impurity="gini",
                                     maxDepth=3,
                                     maxBins=32)

In [62]:
predictions = model.predict(test.map(lambda x:x.features))
predictions.take(5)

[1.0, 1.0, 1.0, 1.0, 1.0]

In [63]:
testPredictins = test.map(lambda x:x.label).zip(predictions)
testPredictins.take(5)

[(1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]

In [64]:
testAccuracy = testPredictins.filter(lambda x:x[0]==x[1]).count()/float(test.count())
print(testAccuracy)

1.0


In [65]:
from pyspark.mllib.evaluation import MulticlassMetrics
metrics = MulticlassMetrics(testPredictins)
metrics.accuracy

1.0

In [66]:
metrics.fMeasure()

1.0

In [67]:
metrics.precision(1.0)

1.0

In [68]:
metrics.confusionMatrix().toArray()

array([[19.,  0.,  0.],
       [ 0., 19.,  0.],
       [ 0.,  0., 18.]])

In [70]:
print(model.toDebugString())

DecisionTreeModel classifier of depth 2 with 5 nodes
  If (feature 0 <= 1.5)
   Predict: 1.0
  Else (feature 0 > 1.5)
   If (feature 0 <= 2.5)
    Predict: 2.0
   Else (feature 0 > 2.5)
    Predict: 3.0

